In [ ]:
# Databricks notebook source
# MAGIC %md
# MAGIC # H3 Feature Engineering for Site Selection Modeling
# MAGIC
# MAGIC Generates H3-8 resolution features by aggregating:
# MAGIC - POI data from OSM
# MAGIC - Demographics from Census block groups (area-weighted)
# MAGIC - Competition data
# MAGIC - Distance features to RMC locations and competitors
# MAGIC - Urbanicity scores
# MAGIC
# MAGIC Output: `h3_features_gold` table in gold schema

In [ ]:
!pip install folium

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from datetime import datetime
import yaml
import folium
import json

# Notebook parameters
dbutils.widgets.text("catalog", "")
dbutils.widgets.text("bronze_schema", "")
dbutils.widgets.text("silver_schema", "")
dbutils.widgets.text("gold_schema", "")
dbutils.widgets.text("state_fips", "")
dbutils.widgets.text("config_path", "")

catalog = dbutils.widgets.get("catalog")
bronze_schema = dbutils.widgets.get("bronze_schema")
silver_schema = dbutils.widgets.get("silver_schema")
gold_schema = dbutils.widgets.get("gold_schema")
state_fips = dbutils.widgets.get("state_fips")
config_path = dbutils.widgets.get("config_path")

assert catalog and bronze_schema and silver_schema and gold_schema and state_fips and config_path, \
    "Missing required parameters"

# Load configuration
with open(config_path, 'r') as f:
    config = yaml.safe_load(f)

H3_RESOLUTION = config['h3_grid']['resolution']
LAT_STEP = config['h3_grid']['grid_sampling']['lat_step']
LON_STEP = config['h3_grid']['grid_sampling']['lon_step']
URBANICITY_WEIGHTS = config['urbanicity']['weights']
NULL_DISTANCE_VALUE = config['distance']['null_value']

In [ ]:
# Load state boundary
state_df = spark.table(f"{catalog}.{bronze_schema}.bronze_census_states") \
    .filter(F.col("state_fips") == state_fips)

In [ ]:
# Generate H3 cells covering state
h3_cells_df = state_df.select(
    F.explode(F.expr(f"h3_polyfillash3string(ST_AsText(geometry), {H3_RESOLUTION})")).alias("h3_cell_id")
).cache()

display(h3_cells_df.limit(5))

In [ ]:
# Get H3 cell boundaries and clip to state
h3_base_df = h3_cells_df.select(
    F.col("h3_cell_id"),
    F.expr("ST_GeomFromGeoJSON(h3_boundaryasgeojson(h3_cell_id))").alias("h3_geometry"),
    F.lit(H3_RESOLUTION).alias("h3_resolution")
)

state_boundary_broadcast = F.broadcast(state_df.select(F.col("geometry").alias("state_geometry")))

h3_base_df = h3_base_df.crossJoin(state_boundary_broadcast) \
    .filter(F.expr("ST_Intersects(h3_geometry, state_geometry)")) \
    .withColumn("h3_geometry", F.expr("ST_Intersection(h3_geometry, state_geometry)")) \
    .withColumn("h3_area_sqkm", F.expr("ST_Area(h3_geometry) / 1000000")) \
    .select("h3_cell_id", "h3_geometry", "h3_resolution", "h3_area_sqkm") \
    .cache()

display(h3_base_df.limit(5))

In [ ]:
h3_sample = h3_base_df.limit(20000)

h3_geojson = h3_sample.withColumn(
    "geojson",
    F.expr("ST_AsGeoJSON(h3_geometry)")
).select("h3_cell_id", "geojson").collect()

features = [
    {
        "type": "Feature",
        "properties": {"h3_cell_id": row["h3_cell_id"]},
        "geometry": json.loads(row["geojson"])
    }
    for row in h3_geojson
]

m = folium.Map(location=[ 42.40, -71.38], zoom_start=6)
folium.GeoJson(
    {"type": "FeatureCollection", "features": features},
    style_function=lambda x: {"fillColor": "blue", "color": "black", "weight": 1, "fillOpacity": 0.3}
).add_to(m)

m

%md
## Step 2: Aggregate POIs to H3 Cells


In [ ]:
# Load POI data
pois_df = spark.table(f"{catalog}.{silver_schema}.silver_osm_pois") \
    .select(
        F.col("latitude"),
        F.col("longitude"),
        F.col("poi_category"),
        F.expr("ST_Point(longitude, latitude, 4326)").alias("poi_point")
    )

In [ ]:
display(pois_df.groupBy("poi_category").agg(F.count("*")))

In [ ]:
display(pois_df.agg(F.count("*")))

In [ ]:
pois_sample_df = pois_df.limit(1000)
poi_geojson = pois_sample_df.withColumn(
    "geojson",
    F.expr("ST_AsGeoJSON(poi_point)")
).select("poi_category", "geojson").collect()

features = [
    {
        "type": "Feature",
        "properties": {"poi_category": row["poi_category"]},
        "geometry": json.loads(row["geojson"])
    }
    for row in poi_geojson
]

m = folium.Map(location=[ 42.40, -71.38], zoom_start=6)
folium.GeoJson(
    {"type": "FeatureCollection", "features": features},
    style_function=lambda x: {"fillColor": "blue", "color": "black", "weight": 1, "fillOpacity": 0.3}
).add_to(m)

m

In [ ]:
# Spatial join POIs to H3 cells and aggregate by category
poi_h3_join = h3_base_df.alias("h3").join(
    pois_df.alias("poi"),
    F.expr("ST_Contains(h3.h3_geometry, poi.poi_point)"),
    "left"
)
display(poi_h3_join.limit(10))

poi_category_agg = poi_h3_join.groupBy("h3_cell_id") \
    .pivot("poi_category") \
    .agg(F.count("poi_point"))

poi_features = poi_category_agg
poi_category_cols = [col for col in poi_features.columns if col != "h3_cell_id"]

poi_features = poi_features.withColumn(
    "total_poi_count",
    sum([F.coalesce(F.col(c), F.lit(0)) for c in poi_category_cols])
)

for col in poi_category_cols:
    poi_features = poi_features.withColumnRenamed(col, f"poi_count_{col}")

poi_count_cols = [col for col in poi_features.columns if col.startswith("poi_count_") or col == "total_poi_count"]
poi_features = poi_features.fillna(0, subset=poi_count_cols)

display(poi_features.limit(5))

In [ ]:
display(poi_features.agg(F.sum(F.col("total_poi_count"))).collect()[0][0])

%md
## Step 3: Aggregate Demographics from Block Groups (Area-Weighted)


In [ ]:
# Load block group geometries and demographics
bg_geom_df = spark.table(f"{catalog}.{bronze_schema}.bronze_census_blockgroups") \
    .select(
        F.col("geoid").alias("bg_geoid"),
        F.col("geometry").alias("bg_geometry")
    )

bg_demo_df = spark.table(f"{catalog}.{bronze_schema}.bronze_census_demographics") \
    .withColumn("geoid", F.concat(F.col("state"), F.col("county"), F.col("tract"), F.col("block_group")))

bg_df = bg_geom_df.join(bg_demo_df, bg_geom_df.bg_geoid == bg_demo_df.geoid, "inner") \
    .drop(bg_demo_df.geoid)

In [ ]:
# Spatial join H3 cells with block groups and calculate intersection ratios
bg_h3_intersect = h3_base_df.alias("h3").join(
    bg_df.alias("bg"),
    F.expr("ST_Intersects(h3.h3_geometry, bg.bg_geometry)"),
    "inner"
)

bg_h3_intersect = bg_h3_intersect.withColumn(
    "intersection_area",
    F.expr("ST_Area(ST_Intersection(bg_geometry, h3_geometry))")
).withColumn(
    "bg_total_area",
    F.expr("ST_Area(bg_geometry)")
).withColumn(
    "intersection_ratio",
    F.col("intersection_area") / F.col("bg_total_area")
).cache()

display(bg_h3_intersect.select("h3_cell_id", "bg_geoid", "intersection_ratio").limit(5))

In [ ]:
# Load demographic variables from config
demo_vars = config['demographic_variables']

pop_vars = demo_vars['population']
income_vars = demo_vars['income']
median_vars = demo_vars['median']
household_vars = demo_vars['households']
education_vars = demo_vars['education']
employment_vars = demo_vars['employment']
housing_vars = demo_vars['housing']
commute_vars = demo_vars['commute']

count_vars = pop_vars + income_vars + household_vars + education_vars + employment_vars + housing_vars + commute_vars

In [ ]:
# Area-weighted aggregation for count variables
existing_count_vars = [v for v in count_vars if v in bg_h3_intersect.columns]

weighted_cols = [F.col("h3_cell_id"), F.col("bg_geoid"), F.col("intersection_ratio")]
weighted_cols.extend([
    (F.coalesce(F.col(var), F.lit(0)) * F.col("intersection_ratio")).alias(f"{var}_weighted")
    for var in existing_count_vars
])

bg_h3_weighted = bg_h3_intersect.select(weighted_cols)

agg_exprs = [
    F.sum(f"{var}_weighted").cast("long").alias(var) 
    for var in existing_count_vars
]

demo_count_features = bg_h3_weighted.groupBy("h3_cell_id").agg(*agg_exprs)
display(demo_count_features.limit(2))

In [ ]:
# Take median and rate variables from largest overlapping block group
existing_median_vars = [v for v in median_vars if v in bg_h3_intersect.columns]

rate_vars = existing_median_vars.copy()
if 'per_capita_income' in bg_h3_intersect.columns:
    rate_vars.append('per_capita_income')

window_spec = Window.partitionBy("h3_cell_id").orderBy(F.desc("intersection_ratio"))

demo_rate_features = bg_h3_intersect.withColumn("rank", F.row_number().over(window_spec)) \
    .filter(F.col("rank") == 1) \
    .select(F.col("h3_cell_id"), *[F.col(v) for v in rate_vars])    

demo_features = demo_count_features.join(demo_rate_features, "h3_cell_id", "outer")

count_cols = [c for c in demo_count_features.columns if c != "h3_cell_id"]
demo_features = demo_features.fillna(0, subset=count_cols)

display(demo_features.limit(5))

In [ ]:
# Load competitor locations
competitors_df = spark.table(f"{catalog}.{bronze_schema}.competitor_locations") \
    .select(
        F.col("latitude"),
        F.col("longitude"),
        F.col("store_type"),
        F.expr("ST_Point(longitude, latitude, 4326)").alias("competitor_point")
    ).cache()

In [ ]:
# Spatial join and aggregate competitors by type
comp_h3_join = h3_base_df.alias("h3").join(
    competitors_df.alias("comp"),
    F.expr("ST_Contains(h3.h3_geometry, comp.competitor_point)"),
    "left"
)

comp_total_agg = comp_h3_join.groupBy("h3_cell_id") \
    .agg(F.count("competitor_point").alias("total_competitor_count"))

comp_brand_agg = comp_h3_join.groupBy("h3_cell_id") \
    .pivot("store_type") \
    .agg(F.count("competitor_point"))

comp_brand_cols = [col for col in comp_brand_agg.columns if col != "h3_cell_id"]
for col in comp_brand_cols:
    new_col_name = f"competitor_count_{col}".replace(" ", "_")
    comp_brand_agg = comp_brand_agg.withColumnRenamed(col, new_col_name)

comp_features = comp_total_agg.join(comp_brand_agg, "h3_cell_id", "left")

comp_count_cols = [col for col in comp_features.columns if col.startswith("competitor_count_") or col == "total_competitor_count"]
comp_features = comp_features.fillna(0, subset=comp_count_cols)

display(comp_features.limit(5))

%md
## Step 5: Calculate Distance Features


In [ ]:
# Calculate H3 cell centers
h3_centers_df = (
    h3_base_df
    .select(
        F.col("h3_cell_id"),
        F.expr("h3_centeraswkt(h3_cell_id)").alias("center_wkt")
    )
    .withColumn(
        "h3_center_point",
        F.expr("ST_GeomFromWKT(center_wkt, 4326)")
    )
    .withColumn(
        "center_lat",
        F.expr("ST_Y(h3_center_point)")
    )
    .withColumn(
        "center_lon",
        F.expr("ST_X(h3_center_point)")
    )
    .select("h3_cell_id", "h3_center_point", "center_lat", "center_lon")
)

if config['performance']['cache_intermediate_results']:
    h3_centers_df = h3_centers_df.cache()

display(h3_centers_df.limit(5))

In [ ]:
# Load RMC locations and calculate distances in miles
rmc_df = spark.table(f"{catalog}.{bronze_schema}.rmc_retail_locations_grocery") \
    .select(
        F.col("latitude"),
        F.col("longitude"),
        F.expr("ST_Point(longitude, latitude, 4326)").alias("rmc_point")
    )

h3_rmc_distances = h3_centers_df.crossJoin(F.broadcast(rmc_df.select("rmc_point"))) \
    .withColumn(
        "distance_miles",
        F.expr("ST_Distance(h3_center_point, rmc_point) / 1609.34")
    )

rmc_distance_features = h3_rmc_distances.groupBy("h3_cell_id") \
    .agg(F.min("distance_miles").alias("distance_to_nearest_rmc_miles"))

display(rmc_distance_features.limit(5))

In [ ]:
# Calculate distances to competitors in miles
h3_comp_distances = h3_centers_df.crossJoin(
    F.broadcast(competitors_df.select("store_type", "competitor_point"))
).withColumn(
    "distance_miles",
    F.expr("ST_Distance(h3_center_point, competitor_point) / 1609.34")
)

# Get minimum distance per H3 cell per brand in single aggregation
comp_distance_agg = h3_comp_distances.groupBy("h3_cell_id", "store_type") \
    .agg(F.min("distance_miles").alias("min_distance_miles"))

# Pivot to get one column per brand
comp_distance_pivot = comp_distance_agg.groupBy("h3_cell_id") \
    .pivot("store_type") \
    .agg(F.first("min_distance_miles"))

# Rename columns to standard format
comp_distance_cols = [col for col in comp_distance_pivot.columns if col != "h3_cell_id"]
for col in comp_distance_cols:
    new_col_name = f"distance_to_{col.lower().replace(' ', '_')}_miles"
    comp_distance_pivot = comp_distance_pivot.withColumnRenamed(col, new_col_name)

# Join RMC and competitor distances
distance_features = rmc_distance_features.join(comp_distance_pivot, "h3_cell_id", "left")

# Fill null distances with configured null value
distance_cols = [col for col in distance_features.columns if col.startswith("distance_to_")]
distance_features = distance_features.fillna(NULL_DISTANCE_VALUE, subset=distance_cols)

display(distance_features.limit(5))

%md
## Step 6: Calculate Urbanicity Score


In [ ]:
# Calculate population density
urbanicity_base = h3_base_df.select("h3_cell_id", "h3_area_sqkm") \
    .join(demo_features.select("h3_cell_id", "total_population"), "h3_cell_id", "left") \
    .join(poi_features.select("h3_cell_id", "total_poi_count"), "h3_cell_id", "left") \
    .fillna(0, subset=["total_population", "total_poi_count"])

display(urbanicity_base.limit(5))

In [ ]:
# Normalize and calculate urbanicity score with deciles and categories
stats = urbanicity_base.agg(
    F.min("total_poi_count").alias("min_poi_count"),
    F.max("total_poi_count").alias("max_poi_count")
).collect()[0]

urbanicity_base = urbanicity_base.withColumn(
    "total_poi_count_norm",
    F.when(
        (F.lit(stats["max_poi_count"]) - F.lit(stats["min_poi_count"])) > 0,
        (F.col("total_poi_count") - F.lit(stats["min_poi_count"])) / (F.lit(stats["max_poi_count"]) - F.lit(stats["min_poi_count"]))
    ).otherwise(0)
).withColumn(
    "urbanicity_score",
    F.lit(URBANICITY_WEIGHTS['poi_count']) * F.col("total_poi_count_norm")
).withColumn(
    "urbanicity_decile",
    F.ntile(10).over(Window.orderBy("urbanicity_score"))
).withColumn(
    "urbanicity_category",
    F.when(F.col("urbanicity_decile") >= 8, F.lit("urban"))
    .when(F.col("urbanicity_decile") >= 4, F.lit("suburban"))
    .otherwise(F.lit("rural"))
)

urbanicity_features = urbanicity_base.select(
    "h3_cell_id",
    "total_poi_count_norm",
    "urbanicity_score",
    "urbanicity_decile",
    "urbanicity_category"
).cache()

display(urbanicity_features.orderBy(F.desc("urbanicity_score")).limit(10))

%md
## Step 7: Join All Features and Write to Silver Table


In [ ]:
# Join all features efficiently
h3_features_silver = h3_base_df.select("h3_cell_id", "h3_geometry", "h3_resolution") \
    .join(poi_features, "h3_cell_id", "left") \
    .join(demo_features, "h3_cell_id", "left") \
    .join(comp_features, "h3_cell_id", "left") \
    .join(distance_features, "h3_cell_id", "left") \
    .join(
        urbanicity_features.select("h3_cell_id", 
                                    "total_poi_count_norm", "urbanicity_score", "urbanicity_decile", "urbanicity_category"),
        "h3_cell_id",
        "left"
    ) \
    .withColumn("processing_timestamp", F.current_timestamp())

# Fill nulls with 0 for numeric columns only
numeric_cols = [
    field.name for field in h3_features_silver.schema.fields 
    if field.dataType.typeName() in ['long', 'double', 'integer', 'float']
    and field.name not in ['h3_resolution']
]
h3_features_silver = h3_features_silver.fillna(0, subset=numeric_cols)

In [ ]:
display(h3_features_silver.limit(10))

In [ ]:
# Write to gold table
output_table = f"{catalog}.{gold_schema}.gold_h3_features"

h3_features_silver.write \
    .format("delta") \
    .mode(config['output']['write_mode']) \
    .option("overwriteSchema", "true") \
    .saveAsTable(output_table)

# Unpersist cached DataFrames
h3_cells_df.unpersist()
h3_base_df.unpersist()
bg_h3_intersect.unpersist()
competitors_df.unpersist()
urbanicity_features.unpersist()

if config['performance']['cache_intermediate_results']:
    h3_centers_df.unpersist()

display(h3_features_silver.limit(10))